# Binomial method

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from project_helpers import payoff

In [2]:
# Defines a lower triangular matrix describing the stock dynamics
def binomial_tree_stock(S, T, sigma, n):
    dt = T / n
    u = np.exp(sigma * np.sqrt(dt))
    d = 1 / u

    stock_tree = np.zeros((n + 1, n + 1))
    stock_tree[0, 0] = S

    for i in range(1, n + 1):
        stock_tree[i, i] = stock_tree[i - 1, i - 1] * u
        for j in range(i, n + 1):
            stock_tree[j, i - 1] = stock_tree[j - 1, i - 1] * d

    return stock_tree


# Returns a lower triangular matrix describing the Bermudan option price dynamics
def binomial_bermudan_option_pricing(S, K, T, r, sigma, n, early_exercise_dates, style):
    dt = T / n
    u = np.exp(sigma * np.sqrt(dt))
    d = 1 / u
    p = (np.exp(r * dt) - d) / (u - d)

    # Initialize option value matrix
    option_values = np.zeros((n + 1, n + 1))

    # Calculate the option values at expiration
    for i in range(n + 1):
        option_values[n][i] = payoff(S * (u ** i) * (d ** (n - i)), K, style)

    # Calculate the option values at earlier exercise dates
    for t in range(n - 1, -1, -1):
        for i in range(t + 1):
            hold_value = np.exp(-r * dt) * (p * option_values[t + 1][i+1] + (1 - p) * option_values[t + 1][i])
            if round(t * dt,2) in early_exercise_dates:
                option_values[t][i] = max(payoff(S * (u ** i) * (d ** (t - i)), K, style), hold_value)
            else:
                option_values[t][i] = hold_value

    return option_values


# Defines a lower triangular matrix describing the dynamics of an American option for the model
def binomial_american_option_pricing(S, K, T, r, sigma, n, style):
    dt = T / n
    u = np.exp(sigma * np.sqrt(dt))
    d = 1 / u
    p = (np.exp(r * dt) - d) / (u - d)

    # Initialize option value matrix
    option_values = np.zeros((n + 1, n + 1))

    # Calculate the option values at expiration
    for i in range(n + 1):
        option_values[n][i] = payoff(S * (u ** i) * (d ** (n - i)), K, style)

    # Calculate the option values at earlier exercise dates
    for t in range(n - 1, -1, -1):
        for i in range(t + 1):
            hold_value = np.exp(-r * dt) * (p * option_values[t + 1][i+1] + (1 - p) * option_values[t + 1][i])
            option_values[t][i] = max(payoff(S * (u ** i) * (d ** (t - i)), K, style), hold_value)

    return option_values


# Defines a lower triangular matrix describing the Delta parameter in all possible states for any option in the binomial model
def binomial_delta(option_values, sigma, T, n, S):
    dt = T / n
    u = np.exp(sigma * np.sqrt(dt))
    d = 1 / u

    delta=np.zeros((n, n))

    for i in range(n):
        for j in range(i + 1):
            delta[i][j] = (option_values[i + 1][j + 1] - option_values[i + 1][j]) / (S * (d ** (i - j)) * (u ** (j + 1)) - S * (d ** (i + 1 - j)) * (u ** j))

    return delta


In [3]:
S0 = 40   # Initial stock price
K = 40   # Strike price
T = 1.0   # Time to maturity
r = 0.06  # Risk-free interest rate
sigma = 0.2  # Volatility
n = 1000   # Number of time steps in the binomial tree
M = 10   # Number of exercise dates
early_exercise_dates = np.linspace(0, T, M + 1)  # List of early exercise dates

# martingale measure parameters
dt = T / n
u = np.exp(sigma * np.sqrt(dt))  # Up movement 
d = 1 / u  # Down movement
p = (np.exp(r * dt) - d) / (u - d)  # Probability of Up movement

In [4]:
bermudan_option_prices = binomial_bermudan_option_pricing(S0, K, T, r, sigma, n, early_exercise_dates, 'put')
american_option_prices = binomial_american_option_pricing(S0, K, T, r, sigma, n, 'put')
print("Bermudan Option Price:", bermudan_option_prices[0][0], "American Option Price:", american_option_prices[0][0])

Bermudan Option Price: 2.2777651246905153 American Option Price: 2.3192782619099632


# Semi-Static Replication via Regress-Later Neural Network

In [2]:
from project_helpers import gen_paths, forward, bs_put, bs_call
from project_network import SemiStaticNet, fitting, pre_training
import tensorflow as tf

## Model implementation

In [3]:
# Compute the continuation value Q for the N paths at time t_{m-1}
def continuation_q(trained_weights, stock, delta_t): # normalizingC is normalizing constant from fitting t+1
    
    w_1 = trained_weights[0]  # first layer weights
    b_1 = trained_weights[1]  # first layer biases
    w_2 = trained_weights[2]  # second layer weights
    b_2 = trained_weights[3][0]  # second layer bias
    p = len(w_2)  # number of hidden nodes in the first hidden layer
    cont = []  # continuation vector

    for s_tm in stock:
        sum_cond_exp = b_2
        cond_exp = 0
        for i in range(p):
            w_i = w_1[0][i]
            b_i = b_1[i] + w_1[0][i]
            omega_i = w_2[i][0]

            if w_i >= 0 and b_i >= 0:
                cond_exp = w_i * forward(s_tm, r, delta_t) + b_i
            elif w_i > 0 > b_i:
                cond_exp = w_i * bs_call(s_tm, -b_i / w_i, delta_t, r, sigma)
            elif w_i < 0 < b_i:
                cond_exp = - w_i * bs_put(s_tm, -b_i / w_i, delta_t, r, sigma)
            elif w_i <= 0 and b_i <= 0:
                cond_exp = 0

            sum_cond_exp += omega_i * cond_exp

        # Discount by the risk-free rate
        cont.append(sum_cond_exp * np.exp(- r * delta_t))

    return cont

In [4]:
def model(S0, K, mu, sigma, N, monitoring_dates, style, optimizer=tf.keras.optimizers.Adamax(learning_rate=0.001), model_weights=None):
    """
    monitoring_dates: t0=0, ..., tM=T
    """
    # generate sample paths of asset S
    sample_pathsS = gen_paths(monitoring_dates, S0, mu, sigma, N)

    # evaluate maturity time option values
    M = len(monitoring_dates) - 1
    option = np.zeros(sample_pathsS.shape)
    option[:, M] = payoff(sample_pathsS[:, M], K, style)

    # fit the model at T, store fitted weights to initialize next regression
    betas = []  # store model weights
    
    # double pre-training because why not
    first_pre_weights = pre_training(sample_pathsS[:, M], option[:, M], 
                                     tf.keras.optimizers.Adam(learning_rate=0.01), 500, 0.8,weights=model_weights)

    second_pre_weights = pre_training(sample_pathsS[:, M], option[:, M],
                                      tf.keras.optimizers.Adam(learning_rate=0.0005), 500, 0.8,weights=first_pre_weights)
    
    # now it gets serious
    fitted_beta = fitting(sample_pathsS[:, M], option[:, M],
                          weights=second_pre_weights, optimizer=optimizer)
    betas.append(fitted_beta)

    # Compute option value at T-1
    h = payoff(sample_pathsS[:, M - 1], K, style)                           # value of exercising now
    time_increments = np.diff(monitoring_dates)
    q = continuation_q(fitted_beta, sample_pathsS[:, M - 1], time_increments[M - 1])   # continuation value
    option[:, M - 1] = np.maximum(h, q)                     # take maximum of both values
    
    # compute option values by backward regression
    for m in range(M - 1, 0, -1):

        # fit new weights, initialise with previous weights
        fitted_beta = fitting(sample_pathsS[:, m], option[:, m], weights=fitted_beta, optimizer=optimizer)

        # compute estimated option value one time step earlier
        h = payoff(sample_pathsS[:, m - 1], K, style)                       # value of exercising now
        q = continuation_q(fitted_beta, sample_pathsS[:, m - 1], time_increments[m - 1])       # continuation value
        option[:, m - 1] = np.maximum(h, q)                  # take maximum of both values

        # append the model weights to the beta list
        betas.append(fitted_beta)
    
    # visualize_fit(betas, option, sample_pathsS, optimizer)

    return betas, option
    

In [5]:
def visualize_fit(trained_weights, option_values, stock_values, optimizer):
    dates = len(option_values[0])

    # Lists to store data for plotting
    actual_values_list = []
    predicted_values_list = []
    rlnn = SemiStaticNet(None, optimizer)
    for i in range(dates - 1):
        rlnn.initialize_parameters(trained_weights[i])
        actual_values = option_values[:, dates - 1 - i]
        predicted_values = np.array(rlnn.predict(stock_values[:, dates - 1 - i]))

        actual_values_list.append(actual_values)
        predicted_values_list.append(predicted_values)

    # Create the plot outside the loop
    plt.figure()
    for i in range(dates - 1):
        plt.plot(stock_values[:, dates - 1 - i], actual_values_list[i], label=f'Option value at {dates - i}-th monitoring date', color='b')
        plt.plot(stock_values[:, dates - 1 - i], predicted_values_list[i], label=f'Regressed option value at {dates - i}-th monitoring date', color='r')

    plt.legend()
    plt.show()

    return

In [6]:
# Simulation parameters initialization
# The asset prices follow a Geometric Brownian Motion
S = 40  # Initial price
mu = 0.06  # Drift
sigma = 0.2  # Volatility
K = 40  # Strike price
r = 0.06  # Risk-free rate
T = 1  # Maturity
M = 10  # Number of monitoring dates
N = 10000  # Number of sample paths
pf_style = 'put'  # Payoff type
monitoring_dates = np.linspace(0, T, M + 1) 

In [ ]:
weights, option_value = model(S, K, mu, sigma, N, monitoring_dates, pf_style, optimizer=tf.keras.optimizers.Adamax(learning_rate=0.001))
v_0 = option_value[:, 0][0]
print(v_0)

Restoring model weights from the end of the best epoch: 5.
Epoch 11: early stopping
Restoring model weights from the end of the best epoch: 51.
Epoch 57: early stopping
Restoring model weights from the end of the best epoch: 41.
Epoch 47: early stopping
Restoring model weights from the end of the best epoch: 26.
Epoch 32: early stopping
Restoring model weights from the end of the best epoch: 31.
Epoch 37: early stopping
Restoring model weights from the end of the best epoch: 35.
Epoch 41: early stopping
Restoring model weights from the end of the best epoch: 17.
Epoch 23: early stopping
Restoring model weights from the end of the best epoch: 74.
Epoch 80: early stopping


In [12]:
# perform a few runs that reuse each others weights
v_0s = []
parameters = weights[0]
for i in range(5):
    parameters, option_value = model(S, K, mu, sigma, N, monitoring_dates, pf_style, optimizer=tf.keras.optimizers.Adamax(learning_rate=0.001), model_weights=parameters)
    parameters = parameters[0] 
    v_0= option_value[0, 0]
    v_0s.append(v_0)
print(v_0s)

Restoring model weights from the end of the best epoch: 520.
Epoch 526: early stopping
Restoring model weights from the end of the best epoch: 181.
Epoch 187: early stopping
Restoring model weights from the end of the best epoch: 231.
Epoch 237: early stopping
Restoring model weights from the end of the best epoch: 271.
Epoch 277: early stopping
Restoring model weights from the end of the best epoch: 211.
Epoch 217: early stopping
Restoring model weights from the end of the best epoch: 214.
Epoch 220: early stopping
Restoring model weights from the end of the best epoch: 203.
Epoch 209: early stopping
Restoring model weights from the end of the best epoch: 298.
Epoch 304: early stopping
Restoring model weights from the end of the best epoch: 255.
Epoch 261: early stopping
Restoring model weights from the end of the best epoch: 356.
Epoch 362: early stopping
Restoring model weights from the end of the best epoch: 377.
Epoch 383: early stopping
Restoring model weights from the end of the

## Upper and lower bounds implementation

In [41]:
def upper_bound(trained_weights, stock_paths, strike, monitoring, style, optimizer):

    sample_size = len(stock_paths[:, 0])
    n_mon = len(monitoring)
    differences = np.diff(monitoring)

    b = np.exp(- r * np.cumsum(differences))
    b = np.insert(b, 0, 1)

    martingale = np.zeros((sample_size, n_mon))
    for m in range(1, n_mon):
        rlnn = SemiStaticNet(trained_weights[- m], optimizer)
        q = continuation_q(trained_weights[- m], stock_paths[:, m - 1], differences[m - 1])
        q_part = np.array(q) * b[m - 1]
        g_part = rlnn.predict(stock_paths[:, m], verbose=0) * b[m]
        
        martingale[:, m] = g_part.reshape(-1) - q_part
    
    for i in range(sample_size):
        martingale[i] = np.cumsum(martingale[i])

    upr = np.zeros(sample_size)
    for i in range(sample_size):
        upr[i] = np.max(payoff(stock_paths[i], strike, style) * b - martingale[i])

    upr = np.mean(upr)
    
    return upr

In [18]:
def lower_bound(trained_weights, stock_paths, strike, monitoring, style):

    sample_size, n_mon = len(stock_paths[:, 0]), len(monitoring)
    differences = np.diff(monitoring)

    # Given the weights of the fitted model, compute all the continuation values Q(S_{t_m}) and compare them with
    # h(S_{t_m}) (for every time, for each sample path) and store in tau the exercising times, i.e. the minimum of
    # the times when h(S_{t_m})>Q(S_{t_m})
    tau = np.full(sample_size, n_mon)
    h_of_s =  payoff(stock_paths[:, n_mon - 1], strike, style)

    for m in range(n_mon - 1):
        s = stock_paths[:, m]  # stock values at time m
        h = payoff(s, strike, style)
        # weights is going to be the outcome of model(), so the weights relative to the first time interval are the
        # ones stored for last
        q = continuation_q(trained_weights[n_mon - m - 2], s, differences[m])
        exceed = np.logical_and(h > q, tau > m)
        tau[exceed] = m
        h_of_s[exceed] = h[exceed]

    discounted_values = np.zeros(sample_size)
    for j in range(sample_size):
        indices = np.arange(tau[j]) - 1   # Indices up to the exercising time
        discounted_values[j] = h_of_s[j] * np.exp(-r * np.sum(differences[indices]))

    lowr = np.sum(discounted_values) / sample_size

    return lowr

In [43]:
stock = gen_paths(monitoring_dates, S, mu, sigma, 1000)

low = lower_bound(weights, stock, K, monitoring_dates, 'put')
print(low)

2.0614631418774674


In [44]:
upr = upper_bound(weights, stock, K, monitoring_dates, 'put', optimizer=tf.keras.optimizers.Adamax(learning_rate=0.001))
print(upr)

32/32 [==============================] - 0s 1ms/step
2.4069137289120053
